# <b>FingerFrameDetection-TF2-train.ipynb</b><br>
このノートブックはFingerFrame検出(<https://github.com/Kazuhito00/FingerFrameDetection-TF2>)のモデル訓練用スクリプトです。<br>
Colaboratoryのハードウェア アクセラレータ設定をGPUにして実行してください。<br><br>
This notebook is a model training script for FingerFrame detection(<https://github.com/Kazuhito00/FingerFrameDetection-TF2>). <br>
Set the hardware accelerator setting of Colaboratory to GPU and execute it.

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf

def random_seed_initialize(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

RANDOM_SEED = 42
random_seed_initialize(RANDOM_SEED)

# <b>環境準備(Colaboratory)</b>

### Google Colaboratory用：GPU残時間確認

In [ ]:
!cat /proc/uptime | awk '{print 0.5 - ($1 /60 /60 /24)"days (" 12 - ($1 / 60 / 60) "h)"}'

### Google Driveマウント

In [ ]:
from google.colab import drive
drive.mount('./gdrive')

# <b>環境準備(Tensorflow Object Detection API)</b>

### Protocol Buffers準備

In [ ]:
!curl -OL https://github.com/google/protobuf/releases/download/v3.2.0/protoc-3.2.0-linux-x86_64.zip
!unzip protoc-3.2.0-linux-x86_64.zip -d protoc3
!sudo mv protoc3/bin/* /usr/local/bin/
!sudo mv protoc3/include/* /usr/local/include/
!rm -rf protoc3 protoc-3.2.0-linux-x86_64.zip

In [ ]:
#protoc設定
!git clone --depth 1 https://github.com/tensorflow/models
%cd /content/models/research
!/usr/local/bin/protoc object_detection/protos/*.proto --python_out=.
%cd /content

### 必要ライブラリインストール

In [ ]:
%cd /content/models/research
!cp /content/models/research/object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd /content

In [ ]:
# インストール成否確認
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

# <b>FingerFrameDetection-TF2リポジトリクローン</b>
本作業以降は「/content/models/research」ディレクトリにて実施します。

In [ ]:
%cd /content/models/research

!git clone https://github.com/Kazuhito00/FingerFrameDetection-TF2.git

###  学習データ/検証データ 分割

In [15]:
original_data_dir = '/content/models/research/FingerFrameDetection-TF2/02_tfrecord'
train_data_dir = '/content/models/research/train_data'
val_data_dir = '/content/models/research/val_data'

In [16]:
# 既存データ削除
import shutil

shutil.rmtree(train_data_dir, ignore_errors=True)
shutil.rmtree(val_data_dir, ignore_errors=True)
os.mkdir(train_data_dir)
os.mkdir(val_data_dir)

In [ ]:
# データ数カウント
file_count = len([name for name in os.listdir(original_data_dir) if os.path.isfile(os.path.join(original_data_dir, name))])
print('File Count :', file_count)

In [18]:
# 学習データ/検証データ 分割
import glob

train_ratio = 0.75

file_list = glob.glob(original_data_dir + '/*')
random_sample_list = random.sample(file_list, file_count)

# ディレクトリへコピー
for index, filepath in enumerate(random_sample_list):
    if index < int(file_count * train_ratio):
        # 学習データ
        shutil.copy2(filepath, train_data_dir)
    else:
        # 検証データ
        shutil.copy2(filepath, val_data_dir)

# <b>学習済モデルダウンロード</b>

In [ ]:
model_dir = '/content/models/research/pretrained_model'

model_name = 'efficientdet_d0_coco17_tpu-32'

!wget -P $model_dir https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt
!wget -P $model_dir 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'$model_name'.tar.gz'
!tar xvzf $model_dir'/'$model_name'.tar.gz' -C $model_dir
!rm $model_dir'/'$model_name'.tar.gz'

# <b>モデル訓練</b>

### 学習モデル保存先ディレクトリ作成

In [20]:
!mkdir '/content/gdrive/My Drive/FingerFrameDetection-TF2-save_model'

### TensorBoard起動

In [21]:
%load_ext tensorboard

In [ ]:
tensorboard --logdir '/content/gdrive/My Drive/FingerFrameDetection-TF2-save_model'

### 学習

In [ ]:
!python object_detection/model_main_tf2.py \
    --pipeline_config_path="/content/models/research/FingerFrameDetection-TF2/03_config/efficientdet_d0_pipeline.config" \
    --model_dir="/content/gdrive/My Drive/FingerFrameDetection-TF2-save_model" \
    --num_train_steps=10000 \
    --alsologtostderr \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps=100

### saved model形式へエクスポート

In [ ]:
!python object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path="/content/models/research/FingerFrameDetection-TF2/03_config/efficientdet_d0_pipeline.config" \
    --trained_checkpoint_dir="/content/gdrive/My Drive/FingerFrameDetection-TF2-save_model" \
    --output_directory="/content/gdrive/My Drive/FingerFrameDetection-TF2-save_model/output"